In [1]:
import os
import spacy
import io
from spacy.matcher import Matcher
import pandas as pd
import re
from nltk.corpus import stopwords
import datetime
from datetime import datetime
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import subprocess
import constants as cs
import docxpy
import glob
import numpy as np
import docx2txt
# load pre-trained model
nlp = spacy.load('en_core_web_sm')
nlp 

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [2]:
%run constants.py

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [3]:
cs

<module 'constants' from 'C:\\Users\\HP\\constants.py'>

In [4]:

# FOR INDIAN RESUME RUN THE BELOW FUNCTION TO EXTRACT MOBILE NUMBER
def extract_mobile_number(text):
    phone= re.findall(r'[8-9]{1}[0-9]{9}',text)
    
    if len(phone) > 10:
        return '+' + phone
    else:
        return phone

def extract_email(text):
        email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", text)
        if email:
            try:
                return email[0].split()[0].strip(';')
            except IndexError:
                return None


In [5]:
# Function to remove punctuation and tokenize the text
def tokenText(extText):
   
    # Remove punctuation marks
    punc = '''!()-[]{};:'"\,.<>/?@#$%^&*_~'''
    for ele in extText:
        if ele in punc:
            puncText = extText.replace(ele, "")
            
    # Tokenize the text and remove stop words
    stop_words = set(stopwords.words('english'))
    puncText.split()
    word_tokens = word_tokenize(puncText)
    TokenizedText = [w for w in word_tokens if not w.lower() in stop_words]
    TokenizedText = []
  
    for w in word_tokens:
        if w not in stop_words:
            TokenizedText.append(w)
    return(TokenizedText)            

# Function to extract Name and contact details
def extract_name(Text):
    name = ''  
    for i in range(0,3):
        name = " ".join([name, Text[i]])
    return(name)


In [6]:
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = ['BE','B.E.', 'B.E', 'BS','B.S','B.Com','BCA',
             'ME','M.E', 'M.E.', 'M.S','B.com','10','10+2','BTECH', 'B.TECH', 
             'M.TECH', 'MTECH', 'SSC', 'HSC', 'C.B.S.E','CBSE','ICSE', 'X', 
             'XII','10th','12th',' 10th',' 12th','Bachelor of Arts in Mathematics',
             'Master of Science in Analytics','Bachelor of Business Administration','Major: Business Management']

def extract_education(text):
    nlp_text = nlp(text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]


    edu = {}
    # Extract education degree
    for index, t in enumerate(nlp_text):
        for tex in t.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex in EDUCATION and tex not in STOPWORDS:
                edu[tex] = t + nlp_text[index + 1]

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education


In [7]:

def extract_skills(resume_text):

        nlp_text = nlp(resume_text)
        noun_chunks = nlp_text.noun_chunks

        # removing stop words and implementing word tokenization
        tokens = [token.text for token in nlp_text if not token.is_stop]
        
        # reading the csv file
        data = pd.read_csv("skills.csv") 
        
        # extract values
        skills = list(data.columns.values)
        
        skillset = []
        
        # check for one-grams (example: python)
        for token in tokens:
            if token.lower() in skills:
                skillset.append(token)
        
        # check for bi-grams and tri-grams (example: machine learning)
        for token in noun_chunks:
            token = token.text.lower().strip()
            if token in skills:
                skillset.append(token)
        
        return [i.capitalize() for i in set([i.lower() for i in skillset])]
    

In [8]:
data = pd.read_csv("skills.csv") 
data

,technical skills,ajenti,django-suit,django-xadmin,flask-admin,flower,grappelli,wooey,algorithms,pypattyrn,...,virtualized networks,network automation,cloud management,ai,salesforce,mango db,math,calculus,product launch,mvp


In [9]:
skills = list(data.columns.values)
skills

['technical skills',
 'ajenti',
 'django-suit',
 'django-xadmin',
 'flask-admin',
 'flower',
 'grappelli',
 'wooey',
 'algorithms',
 'pypattyrn',
 'python-patterns',
 'sortedcontainers',
 'django-simple-captcha',
 'django-simple-spam-blocker',
 'django-compressor',
 'django-pipeline',
 'django-storages',
 'fanstatic',
 'fileconveyor',
 'flask-assets',
 'jinja-assets-compressor',
 'webassets',
 'audiolazy',
 'audioread',
 'beets',
 'dejavu',
 'django-elastic-transcoder',
 'eyed3',
 'id3reader',
 'm3u8',
 'mingus',
 'pyaudioanalysis',
 'pydub',
 'pyechonest',
 'talkbox',
 'timeside',
 'tinytag',
 'authomatic',
 'django-allauth',
 'django-oauth-toolkit',
 'flask-oauthlib',
 'oauthlib',
 'python-oauth2',
 'python-social-auth',
 'rauth',
 'sanction',
 'jose',
 'pyjwt',
 'python-jws',
 'python-jwt',
 'bitbake',
 'buildout',
 'platformio',
 'pybuilder',
 'scons',
 'django-cms',
 'djedi-cms',
 'feincms',
 'kotti',
 'mezzanine',
 'opps',
 'plone',
 'quokka',
 'wagtail',
 'widgy',
 'beaker',
 'd

In [10]:
def string_found(string1, string2):
        if re.search(r"\b" + re.escape(string1) + r"\b", string2):
            return True
        return False

def extract_entity_sections_grad(text):
    '''
    Helper function to extract all the raw text from sections of resume specifically for 
    graduates and undergraduates
    :param text: Raw text of resume
    :return: dictionary of entities
    '''
    text_split = [i.strip() for i in text.split('\n')]
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(cs.RESUME_SECTIONS_GRAD)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in cs.RESUME_SECTIONS_GRAD:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
    return entities 


In [11]:
# Function to extract experience details
def expDetails(Text):
    global sent
   
    Text = Text.split()
   
    for i in range(len(Text)-2):
        Text[i].lower()
        
        if Text[i] ==  'years':
            sent =  Text[i-2] + ' ' + Text[i-1] +' ' + Text[i] +' '+ Text[i+1] +' ' + Text[i+2]
            l = re.findall('\d*\.?\d+',sent)
            for i in l:
                a = float(i)
            return(a)
            return (sent)

def extract_experience(resume_text):
    '''
    Helper function to extract experience from resume text
    :param resume_text: Plain resume text
    :return: list of experience
    '''
    wordnet_lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # word tokenization 
    word_tokens = nltk.word_tokenize(resume_text)

    # remove stop words and lemmatize  
    filtered_sentence = [w for w in word_tokens if not w in stop_words and wordnet_lemmatizer.lemmatize(w) not in stop_words] 
    sent = nltk.pos_tag(filtered_sentence)

    # parse regex
    cp = nltk.RegexpParser('P: {<NNP>+}')
    cs = cp.parse(sent)
    
    # for i in cs.subtrees(filter=lambda x: x.label() == 'P'):
    #     print(i)
    
    test = []
    
    for vp in list(cs.subtrees(filter=lambda x: x.label()=='P')):
        test.append(" ".join([i[0] for i in vp.leaves() if len(vp.leaves()) >= 2]))

    # Search the word 'experience' in the chunk and then print out the text after it
    x = [x[x.lower().index('experience') + 10:] for i, x in enumerate(test) if x and 'experience' in x.lower()]
    return x

def string_found(string1, string2):
        if re.search(r"\b" + re.escape(string1) + r"\b", string2):
            return True
        return False

def get_score(_dict):
    _len = len(_dict)
    if _len >= 5:
        return 1
    elif _len < 5 and _len > 2:
        return 0.5
    elif _len  == 1:
        return 0.2
    else:
        return 0


In [12]:
def extract_competencies(text, experience_list):
    '''
    Helper function to extract competencies from resume text
    :param resume_text: Plain resume text
    :return: dictionary of competencies
    '''
    experience_text = ' '.join(experience_list)
    competency_dict = {}
    score = 0

    percentage = (100 // len(cs.COMPETENCIES.keys()))

    for competency in cs.COMPETENCIES.keys():
        matches = {}
        for item in cs.COMPETENCIES[competency]:
            if string_found(item, experience_text):
                if competency not in competency_dict.keys():
                    match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                    if item not in matches.keys():
                        matches[item] = [match.group(0)]
                    else:
                        for i in match.groups():
                            matches[item].append(i)    
                    competency_dict[competency] = matches
                else:
                    match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                    if item not in matches.keys():
                        matches[item] = [match.group(0)]
                    else:
                        for i in match.groups():
                            matches[item].append(i)
                    competency_dict[competency] = matches
                score += get_score(competency_dict[competency]) * percentage
    
    competency_dict['score'] = score 
    list=list(competency_dict.keys())
    return(list)

In [13]:
def extract_competencies_score(text, experience_list):
        '''
        Helper function to extract competencies from resume text
        :param resume_text: Plain resume text
        :return: dictionary of competencies
        '''
        experience_text = ' '.join(experience_list)
        competency_dict = {}
        score = 0

        percentage = (100 // len(cs.COMPETENCIES.keys()))

        for competency in cs.COMPETENCIES.keys():
            matches = {}
            for item in cs.COMPETENCIES[competency]:
                if string_found(item, experience_text):
                    if competency not in competency_dict.keys():
                        match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                        if item not in matches.keys():
                            matches[item] = [match.group(0)]
                        else:
                            for i in match.groups():
                                matches[item].append(i)    
                        competency_dict[competency] = matches
                    else:
                        match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                        if item not in matches.keys():
                            matches[item] = [match.group(0)]
                        else:
                            for i in match.groups():
                                matches[item].append(i)
                        competency_dict[competency] = matches
                    score += get_score(competency_dict[competency]) * percentage
        
        competency_dict['score'] = score 
        return(competency_dict['score'])

In [14]:
def extract_dob(text):
        
    result1=re.findall(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}",text)
    result2=re.findall(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}",text)           
    result3= re.findall(r"[\d]{1,2} [ADFJMNOSadfjmnos]\w* [\d]{4}",text)
    result4=re.findall(r"([\d]{1,2})\.([\d]{1,2})\.([\d]{4})",text)
                
    l=[result1,result2,result3,result4]
    for i in l:
        if i==[]:
            continue
        else:
            return i


def extract_text_from_docx(path):
    '''
    Helper function to extract plain text from .docx files
    :param doc_path: path to .docx file to be extracted
    :return: string of extracted text
    '''
    try:
        temp = docx2txt.process(path_input)
        return temp
    except KeyError:
        return ' '

In [15]:
def extract_text_from_doc(path):
    '''
    Helper function to extract plain text from .docx files
    :param doc_path: path to .docx file to be extracted
    :return: string of extracted text
    '''
    try:
        temp = doc2txt.process(path)
        return temp
    except KeyError:
        return ' '

In [16]:
def extract_text_from_pdf(path):
    '''
    Helper function to extract plain text from .docx files
    :param doc_path: path to .docx file to be extracted
    :return: string of extracted text
    '''
    try:
        temp = pdf2txt.process(path)
        return temp
    except KeyError:
        return ' '

In [17]:
df = pd.DataFrame(columns=['Name','Mobile No.', 'Email','DOB','Education Qualifications',
                           'Skills','Experience (Years)','Last Position','Competence','competence score'], dtype=object)
df

,Name,Mobile No.,Email,DOB,Education Qualifications,Skills,Experience (Years),Last Position,Competence,competence score


In [18]:
i=0
path_input = r"C:/Users/HP/Downloads/Resumes/Peoplesoft Admin_AnubhavSingh"
if path_input.endswith('.docx'):
    text = extract_text_from_docx(path_input)
    tokText = tokenText(text)
    df.loc[i,'Name']=extract_name(tokText)
    df.loc[i,'Mobile No.']=extract_mobile_number(text)
    df.loc[i,'Email']=extract_email(text)
    df.loc[i,'DOB']=extract_dob(text)
    df.loc[i,'Education Qualifications']=extract_education(text)
    df.loc[i,'Skills']=extract_skills(text)
    df.loc[i,'Experience (Years)']=expDetails(text)
    experience_list1=extract_entity_sections_grad(text)

    if 'experience' in experience_list1:
        i=0
        experience_list=experience_list1['experience']
        df.loc[i,'Last Position']=extract_experience(text)
        df.loc[i,'competence score']=extract_competencies_score(text,experience_list)

    else:
        df.loc[i,'Last Position']='NA'
        df.loc[i,'Competence']='NA'
        df.loc[i,'competence score']='NA'
    i+=1

In [19]:
df

,Name,Mobile No.,Email,DOB,Education Qualifications,Skills,Experience (Years),Last Position,Competence,competence score


In [20]:
from docx import Document

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    paragraphs = [p.text for p in doc.paragraphs]
    return '\n'.join(paragraphs)

In [21]:
import pdfplumber

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
        return text

# Rest of the code remains the same


In [22]:
import PyPDF2

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        num_pages = reader.numPages
        text = ''
        for page in range(num_pages):
            text += reader.getPage(page).extractText()
        return text

In [23]:
import os
from docx import Document

def extract_text_from_docx(file_path):
    document = Document(file_path)
    paragraphs = [p.text for p in document.paragraphs]
    return '\n'.join(paragraphs)

def extract_text_from_pdf(file_path):
    # Code to extract text from PDF goes here
    # Replace this with your own implementation or use a library like PyPDF2 or pdfminer


    df = pd.DataFrame(columns=['Name', 'Mobile No.', 'Email', 'DOB', 'Education Qualifications', 'Skills',
                               'Experience (Years)', 'Last Position', 'Competence', 'competence score'])

In [24]:
from docx import Document

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    paragraphs = [p.text for p in doc.paragraphs]
    return "\n".join(paragraphs)

In [25]:
import subprocess

def extract_text_from_doc(file_path):
    process = subprocess.Popen(['antiword', file_path], stdout=subprocess.PIPE)
    output, _ = process.communicate()
    return output.decode('utf-8')


In [26]:
import PyPDF2

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [27]:
import docx2txt

def extract_text_from_doc(file_path):
    text = docx2txt.process(file_path)
    return text

In [28]:
import os
import glob
import pandas as pd
import docx


# Folder path containing the files
folder_path = r"C:/Users/HP/Downloads/Resumes"

# File extensions to consider
file_extensions = ['pdf', 'docx', 'doc']

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Name', 'Mobile No.', 'Email', 'DOB', 'Education Qualifications',
                           'Skills', 'Experience (Years)', 'Last Position', 'Competence',
                           'competence score'])

# Retrieve file paths matching the extensions
file_paths = []
for ext in file_extensions:
    file_paths.extend(glob.glob(os.path.join(folder_path, f"*.{ext}")))

# Process each file
for i, path_input in enumerate(file_paths):
    try:
        if path_input.endswith('.docx'):
            # Process DOCX files
            doc = docx.Document(path_input)
            paragraphs = [p.text for p in doc.paragraphs]
            text = "\n".join(paragraphs)
        elif path_input.endswith('.doc'):
            # Process DOC files
            doc = docx.Document(path_input)
            paragraphs = [p.text for p in doc.paragraphs]
            text = "\n".join(paragraphs)
        elif path_input.endswith('.pdf'):
            # Process PDF files
            text = extract_text_from_pdf(path_input)
        else:
            # Skip unsupported file formats
            continue

        tokText = tokenText(text)
        df.loc[i, 'Name'] = extract_name(tokText)
        df.loc[i, 'Mobile No.'] = extract_mobile_number(text)
        df.loc[i, 'Email'] = extract_email(text)
        df.loc[i, 'DOB'] = extract_dob(text)
        df.loc[i, 'Education Qualifications'] = extract_education(text)
        df.loc[i, 'Skills'] = extract_skills(text)
        df.loc[i, 'Experience (Years)'] = expDetails(text)
        experience_list1 = extract_entity_sections_grad(text)

        if 'experience' in experience_list1:
            experience_list = experience_list1['experience']
            df.loc[i, 'Last Position'] = extract_experience(text)

            df.loc[i, 'competence score'] = extract_competencies_score(text, experience_list)
        else:
            df.loc[i, 'Last Position'] = 'NA'
            df.loc[i, 'Competence'] = 'NA'
            df.loc[i, 'competence score'] = 'NA'

    except ValueError as e:
        if "file is not a Word file" in str(e):
            # Skip files that are not valid Word files
            print(f"Skipped file: {path_input} - Not a valid Word file")
        else:
            # Handle other ValueErrors
            print(f"Error processing file: {path_input} - {str(e)}")

# Print the resulting DataFrame
print(df)

                                   Name Mobile No. Email             DOB  \
0                  ANIL KUMAR MADDUKURI         []  None            None   
1             Aradhana Tripathi Current         []  None            None   
2                   BUDDHA VAMSI CAREER         []  None    [22-08-1994]   
3                  Chinna Subbarayudu M         []  None            None   
4                           Name : Gopi         []  None    [19-04-1993]   
..                                  ...        ...   ...             ...   
74                        WORKDAY | HCM         []  None            None   
75                   Career objective A         []  None  [12 July 1994]   
76   Venkateswarlu.B Workday Consultant         []  None            None   
77                        Vinay kumar v         []  None            None   
78                        B vinod kumar         []  None            None   

   Education Qualifications  \
0                        []   
1                        

In [29]:
# Assuming the data is stored in a variable called 'df'
data_str = df.to_string(index=False)

# Print the data string
data_str

'                                 Name   Mobile No.                     Email            DOB     Education Qualifications                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Skills Experience (Years)                                                                                                                                                                                          

In [30]:
# Save DataFrame to CSV file
output_file = 'C:/Users/HP/Downloads/Resumes/resumes_data.csv'
output_file

'C:/Users/HP/Downloads/Resumes/resumes_data.csv'

In [31]:
data = df.to_csv("output_file", index=False)
data

In [32]:
resume=pd.read_csv="C:/Users/HP/Downloads/Resumes/resumes_data.csv"
resume

'C:/Users/HP/Downloads/Resumes/resumes_data.csv'

In [33]:
output_file

'C:/Users/HP/Downloads/Resumes/resumes_data.csv'

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("output_file")
data

,Name,Mobile No.,Email,DOB,Education Qualifications,Skills,Experience (Years),Last Position,Competence,competence score
0,ANIL KUMAR MADDUKURI,[],NaN,NaN,[],"['Visual', 'Sql', 'Database', 'Etl', 'Sql serv...",2.40,"[' Currently', ' Project']",NaN,12.0
1,Aradhana Tripathi Current,[],NaN,NaN,[],"['Sales', 'Testing', 'Ecommerce', 'Cloud', 'Sq...",4.00,[],NaN,4.0
2,BUDDHA VAMSI CAREER,[],NaN,['22-08-1994'],[],"['Ibm', 'Engineering', 'Cloud', 'Sql', 'Databa...",2.11,[],NaN,4.0
3,Chinna Subbarayudu M,[],NaN,NaN,[],"['Strategy', 'Testing', 'Sql', 'Database', 'Or...",5.10,[],NaN,0.0
4,Name : Gopi,[],NaN,['19-04-1993'],[],"['Training', 'Testing', 'Cloud', 'Presentation...",3.00,[' XML'],NaN,4.0
...,...,...,...,...,...,...,...,...,...,...
74,WORKDAY | HCM,[],NaN,NaN,[],"['Sales', 'Billing', 'Transactions', 'Rest', '...",6.00,[],NaN,66.0
75,Career objective A,[],NaN,['12 July 1994'],"[('SSC', '2009')]","['Sql server', 'Sql', 'Writing', 'Database', '...",3.20,[' Summary I.T'],NaN,0.0
76,Venkateswarlu.B Workday Consultant,[],NaN,NaN,[],"['Testing', 'Presentation', 'Lifecycle', 'Anal...",5.30,[],NaN,0.0
77,Vinay kumar v,[],NaN,NaN,[],"['Administration', 'Powerpoint', 'Staffing', '...",4.20,"[' Workday Functional Consultant', 'd Report W...",NaN,66.0
